# Корелляции  

Задача: 

**№1.** Используя язык Python, необходимо запрогроммировать модифицированные корреляции для тяжелых и сверхтяжелых нефтей. 

**№2** Построить зависимости вязкости газонасыщенной нефти, газосодержания, сжимаемости и пластовой температуры для различных температур. Исходные данные: газовый фактор, пластовая температура, относительная плотность нефти, относительная плотность газа, давление в сепараторе.

## Границы применимости корреляций


  
  
| Параметр      | Обозначение              | Значение | Размерность |
| :------------- |:------------------:| :-----:| :---------:|
| Плотность нефти     | $$API$$   | [6;22.3] | $$API^0$$ |
| Пластовое давление   | $$P_r$$ |  [1038.49;7411.54] | $$psia$$  |
| Пластовая температура  | $$T_r$$ |  [131.4;250.7] | $$F^0$$  |
| Газосодержание нефти в пластовых условиях    | $$R_s$$   | [17.21;640.25] | $$\frac{scf}{STB}$$ |
| Давление насыщения   | $$API$$   | [208.86;4021.96] | $$psia$$ |
| Давление в сепараторе  | $$API$$   | [14.5;752.2] | $$psia$$ |
| Температура в сепараторе  | $$T_{sp}$$   | [59;177.8] | $$F^0$$ |
| Газосодержание в сепараторе     | $$R_{sp}$$   | [11.1;575.62] | $$\frac{scf}{STB}$$ |
| Газосодержание в поверхностных условиях     | $$R_{st}$$   | [4.39;311.41] | $$\frac{scf}{STB}$$ | 
| Относительная плотность газа на поверхности     | $$\gamma_{g}$$   | [0.623;1.517] | $$$$ |
| Относительная плотность газа в сепараторе     | $$\gamma_{Psp}$$   | [0.623;1.517] | $$$$ |
| Мольная концентрация углекислого газа     | $$YCO2$$   | [0.5;98.8] | _%mol._ |
| Мольная концентрация азота    | $$YN2$$   | [0;63.32] | _%mol._ |
| Мольная концентрация сероводорода    | $$YH2S$$   | [0;1.99] | _%mol._ |
| Объемный коэффициент нефти   | $$OFVF$$   | [1.057;1.362] | $$\frac{bbl}{STB}$$ |
| Cжимаемость нефти    | $$C_o$$   | [3.02;42.9] |  $$psia^{-1} \bullet 10^6$$ |
| Вязкость дегазированной нефти    | $$ \mu_{od}$$   | [7.7;1386.9] | $$cp$$ |
| Вязкость насыщенной нефти    | $$ \mu_{ol}$$   | [2.1;295.9] | $$cp$$ |
| Вязкость недонасыщенной нефти    | $$ \mu_{o}$$   | [2.4;354.6] | $$cp$$ |








## Пункт №1

**Прогроммируем модифицированные корреляции:**

1.Для вычисления давления насыщения в работе используется **Moddified Standing's correlation**:

$$P_b=15.7286\bullet \left[ (\frac{R_s}{\gamma_g}) \bullet \frac{10^{0.0020\bullet T}}{10^{0.0142\bullet API}} \right]$$

Корреляция применима для тяжелых нефтей ($ 10< API^\circ \le22.3 $).

In [2]:
def Bubblepoint_P_HO(Rs, Yg, T, API):
    if 10 < API <= 22.3:
        Pb = 15.7286*((Rs/Yg)**0.7885)*10**(0.0020*T-0.0142*API)
        return Pb
    
    else:
        return 'Корреляция не подходит для данного значения плотности'


print(Bubblepoint_P_HO(231.46,0.696,147.9,11.0))

2112.4282561966543


2.Газонасыщенность:
>Cверхтяжелых нефтей вычисляют с помощью **Modified Standing's correlation**:
$$R_s=\gamma_g\bullet \left(\frac{P_b}{10.7025}\bullet10^{(0.0169\bullet API-0.00156\bullet T)}\right)^{1.1128} $$
>Тяжелых нефтей с помощью **Modifiend Vasquez-Beggs correlation**:
$$R_s= \frac{\gamma_{gcorr} \bullet P_b^{1.2057}}{56.434}\bullet 10^{10.9267\bullet API/(T+460)}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

In [3]:
import math as m


def Solution_GOR(Pb, Yg, T, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Rs = Yg*(Pb/10.7025*10**(0.0169*API-0.00156*T))**1.1128
        return Rs
    
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Rs = Ygcorr*Pb**(1.2057)/56.434*10.9267*(API/(T+460))
        return Rs
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(Solution_GOR(2503.39,0.696,147.9,22,0.679,131.0,58.02))

56.564085705371255


3.Сжимаемость для сверхтяжелых нефтей вычисляют с помощью **Modifiend Vasquez-Beggs correlation**:
$$C_o= \frac{-889.6+3.1374\bullet R_s+20 \bullet T_g-627.3 \bullet \gamma_{gcorr}+41.91 \bullet API}{P_g \bullet 10^5}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

Сжимаемость для тяжелых нефтей вычисляют с помощью **Modifiend Vasquez-Beggs correlation**:
$$C_o= \frac{-2841.8+2.9646\bullet R_s+25.5439 \bullet T_g-1230.5 \bullet \gamma_{gcorr}+41.91 \bullet API}{P_g \bullet 10^5}$$
>Где:
$$\gamma_{gcorr}=\gamma_{gPsp} \bullet \left[1+0.5912 \bullet API \bullet T_{sp} \bullet Log(\frac{P_{sp}}{114.7})10^{-4} \right]$$

In [13]:
import math as m


def Isotherm_Compressibility(Rs, Tg, Pg, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Co = (-889.6+3.1374*Rs+20*Tg-627.3*Ygcorr-81.4476*API)/(Pg*10**(5))
        return Co
    
    elif 10 < API <=  22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*m.log10(Psp/114.7)*10**(-4))
        Co = (-2841.8+2.9646*Rs+25.5439*Tg-1230.5*Ygcorr+41.91*API)/(Pg*10**(5))
        return Co
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(Isotherm_Compressibility(2503.39,0.696,147.9,6.0,0.679,131.0,58.0))

0.0004103918029077642


4.Вязкость дегазированной нефти в находят с помощью следующих корреляций:
>**Modifiend Egbogah-Jack's correlation** для сверхтяжелых нефтей:
$$log\bullet log(\mu_{od} +1)=1.90296-0.012619\bullet API -0.61748 \bullet log(T_g)$$
>**Modifiend Egbogah-Jack's correlation** для тяжелых нефтей:
$$log\bullet log(\mu_{od} +1)=2.06492-0.0179\bullet API -0.70226 \bullet log(T_g)$$

In [5]:
def D_visc(API, T):
    if API <= 10:
        Mod = 10**(10**(1.90296-0.012619*API-0.61748*m.log10(T)))-1
        return Mod
     
    elif 10 < API <= 22.3:
        Mod = 10**(10**(2.06492-0.0179*API-0.70226*m.log10(T)))-1
        return Mod
    
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(D_visc(6.0,147.9))

1177.646405184529


5.Вязкость насыщенной нефти в находят с помощью следующих корреляций:
>**Modifiend Kartoatmodjo's correlation** для сверхтяжелых нефтей:
$$\mu_{ol}=2.3945+0.8927\bullet F+0.001567 \bullet F^2$$
Где
$$F= \left(-0.0335+1.0785\bullet10^{-0.000845\bullet R_s}\right)\bullet\mu_{od}^{(0.5798+0.3432\bullet y)}$$
$$y=10^{-0.00081\bullet R_s}$$
$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$


>**Modifiend Kartoatmodjo's correlation** для тяжелых нефтей:
$$\mu_{ol}=-0.6311+1.078\bullet F-0.003653 \bullet F^2$$
Где
$$F= \left(0.2478+0.6114\bullet10^{-0.000845\bullet R_s}\right)\bullet\mu_{od}^{(0.4731+0.5158\bullet y)}$$
$$y=10^{-0.00081\bullet R_s}$$
$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$

In [6]:
def G_S_visc(Rs, Mod, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (-0.0335+1.0785*10**(-0.000845*Rs))*Mod**(0.5798+0.3432*Y)
        Mol = 2.3945+0.8927*F+0.001567*F**2
        return Mol
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (0.2478+0.6114*10**(-0.000845*Rs))*Mod**(0.4731+0.5158*Y)
        Mol = -0.6311+1.078*F-0.003653*F**2
        return Mol
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(G_S_visc(231.46,1386.9,6.0,0.679,131.0,58.02))

270.78565954057524


6.Вязкость недонасыщенной нефти в находят с помощью следующих корреляций:
>**Modifiend Labedi's correlation** для сверхтяжелых нефтей:
$$\mu_o=\mu_{ol}-\left[(1- \frac{P}{P_b})\bullet \left(\frac{10^{-2.19}\bullet \mu_{od}^{1.055}\bullet P_b^{0.3132}}{10^{0.0099\bullet API}}\right)\right]$$



>**Modifiend Kartoatmodjo's correlation** для тяжелых нефтей:
$$\mu_{o}=0.9886\bullet \mu_{ol}+0.002763\bullet(P-P_b)\bullet(-0.01153\bullet \mu_{ol}^{1.7933}+0.0316\bullet \mu_{ol}^{1.5939})$$
Где

$$\gamma_{gcorr}=\gamma_{gPsp}\bullet \left[1+0.1595\bullet API^{0.4078} \bullet (T_{sp})^{-0.2466}\bullet Log(\frac{P_{sp}}{114.7})\right]$$

In [7]:
def Un_S_visc(Mol, Mod, P, Pb, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Mo = Mol-((1-P/Pb)*10**(-2.19)*Mod**(1.055)*Pb**(0.3132)/10**(0.0099)/API)
        return Mo
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Mo = 0.9886*Mol+0.002763*(P-Pb)*(-0.01153*Mol**(1.7933)+0.0316*Mol**(1.5939))
        return Mo
    else:
        return 'Корреляция не подходит для данного значения плотности'

print(Un_S_visc(295.9,1386.9,3428.75,2503.39,6.0,0.679,131.0,58.02))

305.2112442848726


# Пункт №2
Построим зависимости для t = 30 °С; 50 °С; 70 °С; 90 °С:

𝜇𝑜 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑝𝑏 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑅𝑠 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡); 𝑐𝑜 = 𝑓(𝑝,𝑡 = 𝑐𝑜𝑛𝑠𝑡)

где: 𝜇𝑜 – вязкость газонасыщенной нефти, 𝑝𝑏 – давление насыщения; 𝑅𝑠 – газосодержание; 𝑐𝑜 – сжимаемость нефти

Графики построим при помощи библиотеки plotly.

Учтем, что для t = 30 °С; 50 °С расчеты могут быть некорректны, т.к. данные значения находятся за пределами границ применимости модиффицированных корреляций,для наглядности их визуализировать не будем.

In [95]:
import math as m
import plotly
import plotly.graph_objs as go
import plotly.express as px





#1
def Bubblepoint_P_HO(Rs, Yg, T, API) :
    Pb = 15.7286*((Rs/Yg)**0.7885)*10**(0.0020*T-0.0142*API)
    return Pb

#2
def Solution_GOR(Pb, Yg, T, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Rs = Yg*(Pb/10.7025*10**(0.0169*API-0.00156*T))**1.1128
        return Rs
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*(m.log10(Psp/114.7))*10**(-4))
        Rs = Ygcorr*Pb**(1.2057)/56.434*10.9267*(API/(T+460))
        return Rs
    else:
        return 'Корреляция не подходит для данного значения плотности'

#3
def Isotherm_Compressibility(Rs, Tg, Pg, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*(m.log10(Psp/114.7))*10**(-4))
        Co = (-889.6+3.1374*Rs+20*Tg-627.3*Ygcorr-81.4476*API)/(Pg*10**(5))
        return Co
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.5912*API*Tsp*(m.log10(Psp/114.7))*10**(-4))
        Co = (-2841.8+2.9646*Rs+25.5439*Tg-1230.5*Ygcorr+41.91*API)/(Pg*10**(5))
        return Co
    else:
        return 'Корреляция не подходит для данного значения плотности'

#4
def D_visc(API, T):
    if API <= 10:
        Mod = 10**(10**(1.90296-0.012619*API-0.61748*m.log10(T)))-1
        return Mod
    elif 10 < API <= 22.3:
        Mod = 10**(10**(2.06492-0.0179*API-0.70226*m.log10(T)))-1
        return Mod
    else:
        return 'Корреляция не подходит для данного значения плотности'

#5
def G_S_visc(Rs, Mod, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (-0.0335+1.0785*10**(-0.000845*Rs))*Mod**(0.5798+0.3432*Y)
        Mol = 2.3945+0.8927*F+0.001567*F**2
        return Mol
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Y = 10**(-0.00081*Rs)
        F = (0.2478+0.6114*10**(-0.000845*Rs))*Mod**(0.4731+0.5158*Y)
        Mol = -0.6311+1.078*F-0.003653*F**2
        return Mol
    else:
        return 'Корреляция не подходит для данного значения плотности'

#6
def Un_S_visc(Mol, Mod, P, Pb, API, Ygpsp, Tsp, Psp):
    if API <= 10:
        Mo = Mol-((1-P/Pb)*10**(-2.19)*Mod**(1.055)*Pb**(0.3132)/10**(0.0099)/API)
        return Mo
    elif 10 < API <= 22.3:
        Ygcorr = Ygpsp*(1+0.1595*API**(0.4078)*Tsp**(-0.2466)*m.log10(Psp/114.7))
        Mo = 0.9886*Mol+0.002763*(P-Pb)*(-0.01153*Mol**(1.7933)+0.0316*Mol**(1.5939))
        return Mo
    else:
        return 'Корреляция не подходит для данного значения плотности'

    
#Исходные данные: Газовый фактор, отностительная плотность газа,относительная плотность нефти,
#давление в сепараторе, температура в сепараторе, пластовая температура

#функция для перевода из СИ в имперские единицы измерения
#Td=[30, 50, 70, 90]
#Trr1 = []
#def Perevod(Rrg, SG, Td, Psep):
#    Rsr1 = Rrg * 5.5525
#    Psp1 = 14.696 * Psep
#    APIr1 = 141.5 / SG - 131.5
#    Trr1 = [(i *1.8 + 32) for i in Td]
#
#   return Rsr1, Psp1, APIr1, Trr1

Rsr = 231.45
Ygr = 0.696
Trr = [86,122,158,194]
Ygpspr = 0.679
Pspr = 58.02
APIr = 11.0

#найдем давление насыщения
Pn = Bubblepoint_P_HO(Rsr, Ygr, Trr[2], APIr) 
Pn_si = Pn * 0.06805 #давления насыщения в атмосферах
Pn = m.ceil(Pn)

Pn2 = Pn + 200 #считаю для данного значения, что бы показать, что газосодержание,вязкость, сжимаемость выходят на полку
P = [i for i in range(1, Pn, 5)]

#вязкость дегазированной нефти
Mod30 = D_visc(APIr, Trr[0])
Mod50 = D_visc(APIr, Trr[1])
Mod70 = D_visc(APIr, Trr[2])
Mod90 = D_visc(APIr, Trr[3])

#газосодержание 
Rs30, Rs50, Rs70, Rs90 = [], [], [], []

#сжимаемость
Co30, Co50, Co70, Co90 = [], [], [], []

#вязкость насыщенной нефти
Mol30, Mol50, Mol70, Mol90 = [], [], [], []

for i in P:
    if i >= Pn:
        p = Pn
    else:
        p = i
    Rs30.append(Solution_GOR(p, Ygr, Trr[0], APIr, Ygpspr, Trr[0], Pspr))
    Rs50.append(Solution_GOR(p, Ygr, Trr[1], APIr, Ygpspr, Trr[1], Pspr))
    Rs70.append(Solution_GOR(p, Ygr, Trr[2], APIr, Ygpspr, Trr[2], Pspr))
    Rs90.append(Solution_GOR(p, Ygr, Trr[3], APIr, Ygpspr, Trr[3], Pspr))
    Mol30.append(G_S_visc(Rs30[-1], Mod30, APIr, Ygpspr, Trr[0], Pspr))
    Mol50.append(G_S_visc(Rs50[-1], Mod50, APIr, Ygpspr, Trr[1], Pspr))
    Mol70.append(G_S_visc(Rs70[-1], Mod70, APIr, Ygpspr, Trr[2], Pspr))
    Mol90.append(G_S_visc(Rs90[-1], Mod90, APIr, Ygpspr, Trr[3], Pspr))
    
for p,r in zip(P, Rs30):
    Co30.append(Isotherm_Compressibility(r, Trr[0], p, APIr, Ygpspr, Trr[0], Pspr))

for p,r in zip(P, Rs50):
    Co50.append(Isotherm_Compressibility(r, Trr[1], p, APIr, Ygpspr, Trr[1], Pspr))

for p,r in zip(P, Rs70):
    Co70.append(Isotherm_Compressibility(r, Trr[2], p, APIr, Ygpspr, Trr[2], Pspr))

for p,r in zip(P, Rs90):
    Co90.append(Isotherm_Compressibility(r, Trr[3], p, APIr, Ygpspr, Trr[3], Pspr))
    
#перевод данных в СИ
P_si = [i * 0.06805 for i in P]

Rs30_si = [i * 0.1801 for i in Rs30]
Rs50_si = [i * 0.1801 for i in Rs50]
Rs70_si = [i * 0.1801 for i in Rs70]
Rs90_si = [i * 0.1801 for i in Rs90]

Co30_si = [i * 1.4695 for i in Co30]
Co50_si = [i * 1.4695 for i in Co50]
Co70_si = [i * 1.4695 for i in Co70]
Co90_si = [i * 1.4695 for i in Co90]



In [96]:
#визуализация

print(Pn_si)

fig = go.Figure()

fig.add_trace(go.Scatter(x=P_si, y=Rs70_si, name='$$R_s(T=70^0 C)$$'))
fig.add_trace(go.Scatter(x=P_si, y=Rs90_si, name='$$R_s(T=90^0 C)$$'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость газосодержания от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Газосодержание, м3/м3",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="x+y")
fig.show()




150.5897119048474


In [97]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=P_si, y=Mol70, name='$$M_{ol}(T=70^0 C)$$'))
fig1.add_trace(go.Scatter(x=P_si, y=Mol90, name='$$M_{ol}(T=90^0 C)$$'))
fig1.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость вязкости газонасыщенной нефти от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Вязкость, сП",
                  margin=dict(l=0, r=0, t=30, b=0))
fig1.update_traces(hoverinfo="x+y")
fig1.show()


In [98]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=P_si, y=Co70_si, name='$$C_o(T=70^0 C)$$'))
fig2.add_trace(go.Scatter(x=P_si, y=Co90_si, name='$$C_o(T=90^0 C)$$'))
fig2.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Зависимость сжимаемости от давления",
                  xaxis_title="Давление, атм",
                  yaxis_title="Сжимаемость, 1/атм",
                  margin=dict(l=0, r=0, t=30, b=0))
fig2.update_traces(hoverinfo="x+y")
fig2.show()
